In [1]:
#load os
import os

#load Flask 
#pip install flask==0.12.4
import flask
app = flask.Flask(__name__)
#comment out line before production, only needed during testing:
app.config['TESTING'] = True
from flask import Flask, render_template,request

#load model preprocessing
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras.models
from keras.models import model_from_json
from keras.layers import Input

C:\Users\dlorenz\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


watch: 

https://www.youtube.com/watch?v=MwZwr5Tvyxo 

https://www.youtube.com/watch?v=f6Bf3gl4hWY&t=1743s

https://www.youtube.com/watch?v=IIi6e5oDZ68

https://www.youtube.com/watch?v=RbejfDTHhhg

see code: https://github.com/llSourcell/how_to_deploy_a_keras_model_to_production

In [2]:
# Load pre-trained model into memory
json_file = open('model.json','r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
#load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded Model from disk")

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Loaded Model from disk


In [3]:
# Helper function for tokenizing text to feed through pre-trained deep learning network
def prepDataForDeepLearning(text):
    trainWordFeatures = tokenizer.texts_to_sequences(text)
    textTokenized = pad_sequences(trainWordFeatures, 201, padding='post')
    
    return textTokenized

In [4]:
# Load files needed to create proper matrix using tokens from training data
inputDataTrain = pd.DataFrame(pd.read_csv("train_DrugExp_Text.tsv", "\t", header=None))
trainText = [item[1] for item in inputDataTrain.values.tolist()]
trainingLabels = [0 if item[0] == -1 else 1 for item in inputDataTrain.values.tolist()]

VOCABULARY_SIZE=10000
tokenizer = Tokenizer(num_words=VOCABULARY_SIZE)
tokenizer.fit_on_texts(trainText)

## convert words into word ids
meanLength = np.mean([len(item.split(" ")) for item in trainText])

textTokenized = prepDataForDeepLearning(trainText)

In [5]:
loaded_model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [6]:
#Test that model works based on accuracy in-sample (comment out to run more quickly)
#Note: this model performs at about 82% out-of-sample
loss, accuracy = loaded_model.evaluate(textTokenized,trainingLabels)
print('loss:', loss)
print('accuracy:', accuracy)

In [7]:
#Test with some text (note: lower to zero = more severe):
textDataTest = ['I had a severe reaction to my medication and it was not fun. I developed a severe rash and was not able to sleep. Terrible! I hate the doctor that gave this to me and I am never taking this drug again.']
textTokenizedTest = prepDataForDeepLearning(textDataTest)
#Note: subtract to get things into severity probability:
out = 1-np.asscalar(loaded_model.predict(textTokenizedTest))
out

0.9211631342768669

In [8]:
#Test with some text (note: lower to zero = more severe):
textDataTest = ['I love my medication!']
textTokenizedTest = prepDataForDeepLearning(textDataTest)
#Note: subtract to get things into severity probability:
out = 1-np.asscalar(loaded_model.predict(textTokenizedTest))
out

0.2020358443260193

See above. The first message had a 92% probability being severe and the second had a 20% chance. 
Appears to be working!

In [9]:
# define a predict function as an endpoint 
@app.route('/', methods=['GET','POST'])
def predict():
    #whenever the predict method is called, we're going
    #to input the user entered text into the model
    #and return a prediction
    if request.method=='POST':
        textData = request.form.get('text_entered')
        print(textData)
        textDataArray = [textData]
        print(textDataArray)
        textTokenized = prepDataForDeepLearning(textDataArray)
        print(textTokenized)
        prediction = int((1-np.asscalar(loaded_model.predict(textTokenized)))*100)
        print(prediction)
        #return prediction in new page
        return render_template('prediction.html', prediction=prediction)
    else:
        return render_template("Search_Page.html")   

Note: This code likely will return error message. Follow instructions below to correct error.

You need to edit the echo function definition at ../Lib/site-packages/click/utils.py the default value for the file parameter must be sys.stdout instead of None.

Do the same for the secho function definition at ../Lib/site-packages/click/termui.py

In [10]:
if __name__ == "__main__":
    # start the flask app, allow remote connections
    #decide what port to run the app in
    port = int(os.environ.get('PORT', 5000))
    #this ensures that updates to html/css/js will come through
    app.jinja_env.auto_reload = True  
    #run the app locally on the givn port
    app.run(host='0.0.0.0', port=port) 

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/May/2019 15:06:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [04/May/2019 15:07:36] "POST / HTTP/1.1" 200 -


I had a severe reaction to my medication and it was not fun. I developed a severe rash and was not able to sleep. Terrible! 
['I had a severe reaction to my medication and it was not fun. I developed a severe rash and was not able to sleep. Terrible! ']
[[   1   20    7   83  378    4    5  101    2    6   10   18 1603    1
   564    7   83  470    2   10   18  263    4  109  316    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0